# 前言
本项目对文档进行分类，采用擅长文本分类的朴素贝叶斯算法，通过计算文档中每个单词的 TF-IDF 值，取 TF-IDF 值最高的单词作为区分，从而达到分类文档的目的。

In [1]:
import os
import jieba  # 中文分词工具
from sklearn.feature_extraction.text import TfidfVectorizer  # 计算单词 TF-IDF 值
from sklearn.naive_bayes import MultinomialNB  # 多项式朴素贝叶斯
from sklearn import metrics  # 评估函数

## 加载停用词

In [2]:
# 加载停用词，避免编码失败采用二进制方式读取
with open(r"text classification\stop\stopword.txt",'rb') as f:
    stop_words = [line.strip() for line in f.readlines()]

stop_words[:10]  # 查看读取结果的前10个

[b'\xef\xbb\xbf,',
 b'?',
 b'\xe3\x80\x81',
 b'\xe3\x80\x82',
 b'\xe2\x80\x9c',
 b'\xe2\x80\x9d',
 b'\xe3\x80\x8a',
 b'\xe3\x80\x8b',
 b'\xef\xbc\x81',
 b'\xef\xbc\x8c']

## 加载数据

In [3]:
# 定义函数，加载数据
def load_data(base_path):
    '''
    base_path: 基础路径
    '''

    documents = []  # 分词列表
    labels = []     # 标签列表

    # 循环所有文件并进行分词打标
    for path, folder, files in os.walk(base_path):  # 路径、文件夹、文件
        for file in files:
            label = path.split('\\')[-1]  # 转义
            labels.append(label)
            filename = os.path.join(path, file)   # 把路径和文件名合成一个完整路径
            with open(filename, 'rb') as f:  # 用二进制方式读取
                content = f.read()
                words = list(jieba.cut(content))
                documents.append(' '.join(words))

    return documents, labels

## 构造模型，评估准确率

In [4]:
# 定义函数，构造模型并计算测试集准确率
def train_fun(train_data, train_label, test_data, test_label):
    '''
    train_data: 训练集数据
    train_label: 训练集标签
    test_data: 测试集数据
    test_label: 测试集标签
    '''

    # 创建训练集 TfidfVectorizer 类，max_df=0.5 代表一个单词在 50% 的文档中都出现过,不作为分词统计
    train_tf = TfidfVectorizer(stop_words=stop_words, max_df=0.5)

    # 训练集拟合，得到训练集 TF-IDF 特征空间 features
    train_features = train_tf.fit_transform(train_data)

    # 构造多项式朴素贝叶斯分类器，alpha 越小，迭代次数越多，精度越高
    clf = MultinomialNB(alpha=0.001).fit(train_features, train_label)

    # 创建测试集 TfidfVectorizer 类，词汇表来自训练集tf
    test_tf = TfidfVectorizer(stop_words=stop_words,
                              max_df=0.5,
                              vocabulary=train_tf.vocabulary_)
    # 测试集拟合，得到测试集 TF-IDF 特征空间 test_features
    test_features = test_tf.fit_transform(test_data)

    # 测试集预测
    predicted_labels = clf.predict(test_features)

    # 评估准确率
    score = metrics.accuracy_score(test_label, predicted_labels)

    return score

## 加载数据，评估模型

In [5]:
# 加载训练集数据
train_documents, train_labels = load_data(r"text classification\train")

# 加载测试集数据
test_documents, test_labels = load_data(r"text classification\test")

# 评估模型
score = train_fun(train_documents, train_labels, test_documents, test_labels)

print(score)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\鹏鹏\AppData\Local\Temp\jieba.cache
Loading model cost 0.620 seconds.
Prefix dict has been built successfully.


0.91


说明文档分类有91%的准确率

# 总结

本项目是用贝叶斯分类器对中文文档进行分类，将文档分成四大类：女性、体育、文学、校园。采用中文分词工具 jieba ，这个工具可以对中文文档提取分词，基于这些分词，我们得到分词的权重，即特征矩阵。通过特征矩阵与分类结果，我们创建出朴素贝叶斯分类器，然后用分类器进行预测，最后将预测结果与实际结果做对比，可以得到分类器在测试集上的准确率。

文档分类的原理是通过计算文档中单词的 TF-IDF 值，TF-IDF 值高，意味着这个单词在一个文档中出现的次数多，同时又很少出现在其他文档中。因此这样的单词适合用于分类。另外，在中文文档里，“你，我，并且，一切，所以”等等这些词在分类中没有用，起不到分类作用，为了节省内存和计算时间，我们不计算这些词的 TF-IDF 值，把这些词作为停用词 stop words， TfidfVectorizer 工具就会自动忽略掉这些词，不会计算这些词的 TF-IDF 值。

通过模型预测，可以发现，在这个项目里，模型的预测准确率有 91%，说明模型预测准确率比较高。